In [ ]:
import redis
import geopandas as gpd
import pandas as pd
import os

db = redis.Redis(
    host='redis-14849.c8.us-east-1-3.ec2.cloud.redislabs.com',
    port=14849,
    password='oQTydUmvHPLx4xVUxG3cWtyTyjWq0VJw',
    db=0
)


In [ ]:
# Wczytywanie stacji
eff = gpd.read_file("Dane_administracyjne/effacility.geojson")

# Konwersja do lat/lon (WSG84)
eff = eff.to_crs(4326)

In [ ]:
# Zapisanie danyc stacji do Redis
for _, row in eff.iterrows():
    key = f"stacja:{row['ifcid']}"

    db.hset(key, mapping={
        "id": row["ifcid"],
        "name": row["name1"],           
        "type": row["additional"],       
        "lat": row.geometry.y,
        "lon": row.geometry.x,
    })

In [ ]:
# Definicja parametrów meteorologicznych
parameters = {
    "B00300S": "air_temperature",
    "B00305A": "ground_temperature",
    "B00202A": "wind_direction",
    "B00702A": "wind_speed_10min",
    "B00703A": "wind_max",
    "B00608S": "precip_10min",
    "B00604S": "precip_daily",
    "B00606S": "precip_hour",
    "B00802A": "relative_humidity",
    "B00714A": "max_gust_10min",
    "B00910A": "snow_water_equivalent"
}

In [ ]:
# Wczytanie i zapis statystyk meteo do Redis
for param_code in parameters:
    path = f"wyniki_analizy/station_stats_{param_code}.csv"

    if not os.path.exists(path):
        print(f"⚠ Brak pliku: {path}")
        continue

    stats = pd.read_csv(path)

    # zapisz każdy rekord statystyk
    for _, r in stats.iterrows():
        key = f"meteo:{r['KodSH']}:{r['date']}:{param_code}"

        db.hset(key, mapping={
            "daynight": str(r["daynight"]),
            "mean": float(r["mean"]) if not pd.isna(r["mean"]) else None,
            "median": float(r["median"]) if not pd.isna(r["median"]) else None,
            "trimmed_mean": float(r["trimmed_mean"]) if not pd.isna(r["trimmed_mean"]) else None,
            "count": int(r["count"]) if not pd.isna(r["count"]) else 0,
        })

In [ ]:
# Odczyt danych stacji
station = db.hgetall("stacja:149180070")
station = {k.decode(): v.decode() for k, v in station.items()}
print(station)

In [ ]:
# Odczyt danych meteorologicznych
key = "meteo:249180010:2024-01-29:B00300S"
vals = db.hgetall(key)
vals = {k.decode(): v.decode() for k, v in vals.items()}
print(vals)

In [ ]:
# Odczyt zbiorczy - przykład
def load_stats_from_redis(param):
    keys = db.keys(f"meteo:*:*:{param}")
    rows = []
    for key in keys:
        key_str = key.decode()
        _, station, date, par = key_str.split(":")

        h = db.hgetall(key)
        h = {k.decode(): v.decode() for k,v in h.items()}

        rows.append({
            "KodSH": station,
            "date": date,
            "daynight": h["daynight"],
            "mean": float(h["mean"]),
            "median": float(h["median"]),
            "trimmed_mean": float(h["trimmed_mean"]),
            "count": int(h["count"])
        })

    return pd.DataFrame(rows)

df = load_stats_from_redis("B00300S")

print("\n=== Odczytane dane z Redis ===")
print(df.head(10))
